In [1]:
from math import sqrt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation

In [2]:
# Reading ratings file
# train_ratings = pd.read_excel("/content/drive/MyDrive/test_data.xlsx", usecols=['userId','movieId','rating','timestamp_ratings'])

# Reading movies file
# train_movies = pd.read_excel("/content/drive/MyDrive/test_data.xlsx", usecols=['movieId','title','genres_x'])

In [3]:

df_movies = pd.read_csv(r"C:\Users\Hazel Tan\Documents\BT4222\datasets_BT4222\train_data.csv", usecols=['movieId','title','genres_x'])
df_ratings = pd.read_csv(r"C:\Users\Hazel Tan\Documents\BT4222\datasets_BT4222\train_data.csv", usecols=['userId','movieId','rating','timestamp'])

In [4]:
df_movies

,movieId,title,genres_x
0,2249,My Blue Heaven,Comedy
1,160438,Jason Bourne,Action
2,8810,AVP: Alien vs. Predator,Action|Horror|Sci-Fi|Thriller
3,2018,Bambi,Animation|Children|Drama
4,1645,The Devil's Advocate,Drama|Mystery|Thriller
...,...,...,...
286359,139385,The Revenant,Adventure|Drama
286360,90866,Hugo,Children|Drama|Mystery
286361,8169,*batteries not included,Children|Comedy|Fantasy|Sci-Fi
286362,1292,Being There,Comedy|Drama


In [5]:
df_ratings

,userId,movieId,rating,timestamp
0,126180,2249,2.5,1150714271
1,40636,160438,2.0,1553878417
2,47628,8810,3.5,1107559253
3,109731,2018,4.0,1025848511
4,93451,1645,3.5,1121352601
...,...,...,...,...
286359,49403,139385,2.5,1505950963
286360,137293,90866,3.0,1330138811
286361,81216,8169,3.0,1551306328
286362,111258,1292,3.5,1544936052


In [32]:
df_ratings.userId=df_ratings.userId.apply(lambda x: str())

In [33]:
type(df_ratings.userId)

pandas.core.series.Series

In [6]:
df_movies_ratings = pd.read_csv(r"C:\Users\Hazel Tan\Documents\BT4222\datasets_BT4222\train_data.csv", usecols=['movieId','title','genres_x','userId','rating','timestamp'])
df_movies_ratings

,userId,movieId,rating,timestamp,title,genres_x
0,126180,2249,2.5,1150714271,My Blue Heaven,Comedy
1,40636,160438,2.0,1553878417,Jason Bourne,Action
2,47628,8810,3.5,1107559253,AVP: Alien vs. Predator,Action|Horror|Sci-Fi|Thriller
3,109731,2018,4.0,1025848511,Bambi,Animation|Children|Drama
4,93451,1645,3.5,1121352601,The Devil's Advocate,Drama|Mystery|Thriller
...,...,...,...,...,...,...
286359,49403,139385,2.5,1505950963,The Revenant,Adventure|Drama
286360,137293,90866,3.0,1330138811,Hugo,Children|Drama|Mystery
286361,81216,8169,3.0,1551306328,*batteries not included,Children|Comedy|Fantasy|Sci-Fi
286362,111258,1292,3.5,1544936052,Being There,Comedy|Drama


In [7]:
ratings_matrix_items = df_movies_ratings.pivot_table(index=['movieId'],columns=['userId'],values='rating').reset_index(drop=True)
ratings_matrix_items.fillna( 0, inplace = True )
ratings_matrix_items.shape

(2073, 188)

In [8]:
ratings_matrix_items

userId,1748,1920,2177,2982,3150,8619,9620,11952,14831,17474,...,150005,151214,152107,152690,156183,156480,159816,160747,162047,162516
0,4.5,3.5,0.0,0.0,3.0,0.0,4.0,4.5,4.5,0.0,...,0.0,3.0,5.0,3.0,5.0,4.0,4.0,4.0,3.0,4.5
1,3.0,3.5,3.0,2.0,4.0,2.5,4.0,3.5,0.0,4.0,...,2.5,1.0,3.0,0.0,3.5,0.0,3.0,0.0,0.0,2.5
2,2.5,0.0,0.0,0.0,0.0,2.5,3.0,1.0,3.0,3.0,...,0.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,1.5,0.0
3,0.0,0.0,4.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,4.0,0.0,3.0,2.5,0.0,2.0,0.0,3.0,0.0,4.5,...,0.0,0.0,0.5,0.0,3.0,0.0,3.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2068,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5
2069,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
2070,4.5,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2071,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5


In [9]:
ratings_matrix_items.sum()

userId
1748      3364.5
1920      3067.5
2177      3959.5
2982      3194.0
3150      3109.0
           ...  
156480    2846.5
159816    3783.5
160747    3169.0
162047    3402.0
162516    4673.5
Length: 188, dtype: float64

In [45]:
movie_similarity = 1 - pairwise_distances( ratings_matrix_items.values, metric="cosine" )
np.fill_diagonal( movie_similarity, 0 ) #Filling diagonals with 0s for future use when sorting is done
ratings_matrix_items = pd.DataFrame( movie_similarity )
ratings_matrix_items

,0,1,2,3,4,5,6,7,8,9,...,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072
0,0.000000,0.999496,0.999503,0.999434,0.999493,0.999508,0.999499,0.999487,0.999507,0.999501,...,0.999406,0.999407,0.999367,0.999362,0.999347,0.999376,0.999371,0.999394,0.999425,0.999386
1,0.999496,0.000000,0.999506,0.999402,0.999499,0.999498,0.999486,0.999497,0.999503,0.999506,...,0.999401,0.999435,0.999400,0.999408,0.999398,0.999420,0.999402,0.999430,0.999421,0.999412
2,0.999503,0.999506,0.000000,0.999432,0.999503,0.999504,0.999499,0.999502,0.999508,0.999509,...,0.999388,0.999411,0.999371,0.999378,0.999365,0.999391,0.999375,0.999404,0.999411,0.999387
3,0.999434,0.999402,0.999432,0.000000,0.999447,0.999423,0.999469,0.999436,0.999430,0.999427,...,0.999180,0.999181,0.999116,0.999118,0.999098,0.999140,0.999120,0.999168,0.999214,0.999145
4,0.999493,0.999499,0.999503,0.999447,0.000000,0.999493,0.999499,0.999505,0.999500,0.999505,...,0.999347,0.999380,0.999336,0.999346,0.999335,0.999362,0.999338,0.999375,0.999372,0.999351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2068,0.999376,0.999420,0.999391,0.999140,0.999362,0.999389,0.999323,0.999361,0.999391,0.999395,...,0.999436,0.999485,0.999481,0.999488,0.999487,0.000000,0.999481,0.999492,0.999445,0.999477
2069,0.999371,0.999402,0.999375,0.999120,0.999338,0.999384,0.999309,0.999334,0.999381,0.999378,...,0.999460,0.999485,0.999485,0.999480,0.999478,0.999481,0.000000,0.999485,0.999466,0.999485
2070,0.999394,0.999430,0.999404,0.999168,0.999375,0.999405,0.999343,0.999372,0.999406,0.999406,...,0.999451,0.999490,0.999484,0.999488,0.999486,0.999492,0.999485,0.000000,0.999457,0.999485
2071,0.999425,0.999421,0.999411,0.999214,0.999372,0.999433,0.999368,0.999358,0.999424,0.999404,...,0.999487,0.999471,0.999464,0.999442,0.999432,0.999445,0.999466,0.999457,0.000000,0.999475


In [11]:
def item_similarity(movieName): 
    """
    recomendates similar movies
   :param data: name of the movie 
   """
    try:
        #user_inp=input('Enter the reference movie title based on which recommendations are to be made: ')
        user_inp=movieName
        inp=df_movies[df_movies['title']==user_inp].index.tolist()
        inp=inp[0]

        df_movies['similarity'] = ratings_matrix_items.iloc[inp]
        df_movies.columns = ['movie_id', 'title', 'release_date','similarity']
    except:
        print("Sorry, the movie is not in the database!")

In [12]:
def recommendedMoviesAsperItemSimilarity(user_id):
    """
     Recommending movie which user hasn't watched as per Item Similarity
    :param user_id: user_id to whom movie needs to be recommended
    :return: movieIds to user 
    """
    user_movie= df_movies_ratings[(df_movies_ratings.userId==user_id) & df_movies_ratings.rating.isin([5.0,4.5])][['title']]
    # print(user_movie)
    user_movie=user_movie.iloc[0,0]
    # print(user_movie)
    item_similarity(user_movie)
    sorted_movies_as_per_userChoice=df_movies.sort_values( ["similarity"], ascending = False )
    # print(sorted_movies_as_per_userChoice)
    sorted_movies_as_per_userChoice=sorted_movies_as_per_userChoice[sorted_movies_as_per_userChoice['similarity'] >=0.45]['movie_id']
    # print(sorted_movies_as_per_userChoice)
    recommended_movies=list()
    df_recommended_item=pd.DataFrame()
    user2Movies= df_ratings[df_ratings['userId']== user_id]['movieId']
    # print(user2Movies)
    for movieId in sorted_movies_as_per_userChoice:
            # print(movieId)
            if movieId not in user2Movies:
                # print(movieId)
                df_new= df_ratings[(df_ratings.movieId==movieId)]
                # print(df_new)
                df_recommended_item=pd.concat([df_recommended_item,df_new])
            best10=df_recommended_item.sort_values(["rating"], ascending = False )[1:10] 
    return best10['movieId']

In [13]:
def movieIdToTitle(listMovieIDs):
    """
     Converting movieId to titles
    :param user_id: List of movies
    :return: movie titles
    """
    movie_titles= set()
    for id in listMovieIDs:
        movie_titles.add(df_movies[df_movies['movie_id']==id]['title'].unique()[0])
    return movie_titles

In [14]:
user_id=1748
movieIdToTitle(recommendedMoviesAsperItemSimilarity(user_id))
# print("Recommended movies,:\n",movieIdToTitle(recommendedMoviesAsperItemSimilarity(user_id)))

{'Akira',
 'Beauty and the Beast',
 'North by Northwest',
 'Spellbound',
 'Total Recall'}

In [34]:
ratings_matrix_users = df_movies_ratings.pivot_table(index=['userId'],columns=['movieId'],values='rating').reset_index(drop=True)
ratings_matrix_users.fillna( 0, inplace = True )
movie_similarity = 1 - pairwise_distances( ratings_matrix_users.values, metric="cosine" )
np.fill_diagonal( movie_similarity, 0 ) #Filling diagonals with 0s for future use when sorting is done
ratings_matrix_users = pd.DataFrame( movie_similarity )
ratings_matrix_users

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0.000000,0.548267,0.568642,0.504554,0.566822,0.532100,0.596568,0.553005,0.525475,0.471402,...,0.539998,0.500210,0.555773,0.578303,0.505709,0.449474,0.563255,0.507075,0.516558,0.611842
1,0.548267,0.000000,0.488204,0.477488,0.608433,0.509052,0.540724,0.564231,0.443346,0.622190,...,0.494715,0.430610,0.541512,0.542820,0.555267,0.483651,0.537983,0.544152,0.578244,0.627185
2,0.568642,0.488204,0.000000,0.640191,0.455358,0.672703,0.646190,0.558346,0.647440,0.428688,...,0.624512,0.642085,0.647140,0.647411,0.433148,0.395275,0.614695,0.535773,0.525580,0.611026
3,0.504554,0.477488,0.640191,0.000000,0.419785,0.639102,0.552249,0.539383,0.624760,0.428400,...,0.577657,0.650748,0.612875,0.629929,0.410693,0.404565,0.551264,0.540558,0.503595,0.621715
4,0.566822,0.608433,0.455358,0.419785,0.000000,0.451348,0.506919,0.544528,0.411650,0.548933,...,0.478945,0.399301,0.478557,0.492984,0.555632,0.479881,0.531303,0.478473,0.541056,0.577325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,0.449474,0.483651,0.395275,0.404565,0.479881,0.461894,0.455206,0.395580,0.460316,0.424640,...,0.494015,0.384892,0.483838,0.461884,0.510710,0.000000,0.468626,0.563018,0.522645,0.566384
184,0.563255,0.537983,0.614695,0.551264,0.531303,0.624083,0.600376,0.558692,0.614050,0.475970,...,0.602889,0.553407,0.641138,0.626418,0.520271,0.468626,0.000000,0.571250,0.590637,0.625502
185,0.507075,0.544152,0.535773,0.540558,0.478473,0.603931,0.594778,0.472457,0.577156,0.471989,...,0.606020,0.518571,0.620349,0.568480,0.493944,0.563018,0.571250,0.000000,0.605166,0.647012
186,0.516558,0.578244,0.525580,0.503595,0.541056,0.586468,0.544654,0.494579,0.542272,0.482926,...,0.595030,0.508131,0.591377,0.580451,0.557566,0.522645,0.590637,0.605166,0.000000,0.639598


In [16]:
similar_user_series= ratings_matrix_users.idxmax(axis=1)
df_similar_user= similar_user_series.to_frame()

In [17]:
df_similar_user.columns=['similarUser']

In [18]:
df_similar_user

,similarUser
0,110
1,32
2,98
3,68
4,32
...,...
183,89
184,16
185,89
186,114


In [65]:
df_ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [40]:
def user_similarity(movieName): 
    """
    recomendates similar movies
   :param data: name of the movie 
   """
    try:
        #user_inp=input('Enter the reference movie title based on which recommendations are to be made: ')
        user_inp=movieName
        inp=df_movies_ratings[df_movies_ratings['title']==user_inp].index.tolist()
        inp=inp[0]
        # print(inp)

        df_ratings['similarity'] = ratings_matrix_users.iloc[inp]
        
        df_ratings.columns = ['userId', 'movieId', 'rating','timestamp','similarity']
    except:
        print("Sorry, the movie is not in the database!")

In [41]:
def getRecommendedMoviesAsperUserSimilarity(user_id):
    """
     Recommending movie which user hasn't watched as per Item Similarity
    :param user_id: user_id to whom movie needs to be recommended
    :return: movieIds to user 
    """
    # user2Movies= df_ratings[df_ratings['userId']== userId]['movieId']
    user_movie= df_movies_ratings[(df_movies_ratings.userId==user_id) & df_movies_ratings.rating.isin([5.0,4.5])][['title']]
    # print(user_movie)
    # sim_user=df_similar_user.iloc[0,0]
    user_movie=user_movie.iloc[0,0]
    # print(user_movie)
    user_similarity(user_movie)
    sorted_users_as_per_movieChoice=df_movies.sort_values( ["similarity"], ascending = False )
    # print(sorted_movies_as_per_userChoice)
    sorted_users_as_per_movieChoice=sorted_users_as_per_movieChoice[sorted_users_as_per_movieChoice['similarity'] >=0.45]['movie_id']
    # print(sorted_movies_as_per_userChoice)
    recommended_movies=list()
    df_recommended_item=pd.DataFrame()
    user2Movies= df_ratings[df_ratings['userId']== user_id]['movieId']
    # print(user2Movies)
    for movieId in sorted_users_as_per_movieChoice:
            # print(movieId)
            if movieId not in user2Movies:
                # print(movieId)
                df_new= df_ratings[(df_ratings.movieId==movieId)]
                # df_new= df_movies_ratings[(df_movies_ratings.userId==sim_user) & (df_movies_ratings.movieId==movieId)]
                # print(df_new)
                df_recommended_item=pd.concat([df_recommended_item,df_new])
            best10=df_recommended_item.sort_values(["rating"], ascending = False )[1:10] 
    return best10['movieId']

In [42]:
# movieId_recommended=list()
# def getRecommendedMoviesAsperUserSimilarity(userId):
#     """
#      Recommending movies which user hasn't watched as per User Similarity
#     :param user_id: user_id to whom movie needs to be recommended
#     :return: movieIds to user 
#     """
#     user2Movies= df_ratings[df_ratings['userId']== userId]['movieId']
#     print(user2Movies)
#     sim_user=df_similar_user.iloc[0,0]
#     print(sim_user)
#     df_recommended=pd.DataFrame(columns=['movieId','title','genres','userId','rating','timestamp'])
#     for movieId in df_ratings[df_ratings['userId']== sim_user]['movieId']:
#         if movieId not in user2Movies:
#             df_new= df_movies_ratings[(df_movies_ratings.userId==sim_user) & (df_movies_ratings.movieId==movieId)]
#             df_recommended=pd.concat([df_recommended,df_new])
#         best10=df_recommended.sort_values(['rating'], ascending = False )[1:10]  
#     return best10['movieId']

In [43]:
user_id=1748
recommend_movies= movieIdToTitle(getRecommendedMoviesAsperUserSimilarity(user_id))
print("Movies you should watch are:\n")
print(recommend_movies)

163
Movies you should watch are:

{'Taxi Driver', 'Dancer in the Dark', 'Superman'}


In [38]:
def get_user_similar_movies( user1, user2 ):
    
    """
     Returning common movies and ratings of same for both the users
    :param user1,user2: user ids of 2 users need to compare
    :return: movieIds to user 
    """
    common_movies = df_movies_ratings[df_movies_ratings.userId == user1].merge(
      df_movies_ratings[df_movies_ratings.userId == user2],
      on = "movieId",
      how = "inner" )
    common_movies.drop(['movieId','genres_x_x','genres_x_y', 'timestamp_x','timestamp_y','title_y'],axis=1,inplace=True)
    return common_movies

In [39]:
get_user_similar_movies(1748,1920)

,userId_x,rating_x,title_x,userId_y,rating_y
0,1748,3.0,Inside Out,1920,4.0
1,1748,3.0,Inside Out,1920,4.0
2,1748,3.0,Inside Out,1920,4.0
3,1748,3.0,Inside Out,1920,4.0
4,1748,3.0,Inside Out,1920,4.0
...,...,...,...,...,...
2172,1748,4.5,Good Will Hunting,1920,4.0
2173,1748,4.0,Braveheart,1920,3.5
2174,1748,4.5,Kingsman: The Secret Service,1920,3.5
2175,1748,3.0,Nick of Time,1920,3.0


PANGGANG

In [49]:
n_users = df_ratings.userId.unique().shape[0]
n_movies = df_ratings.movieId.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 188 | Number of movies = 2073


In [50]:
Ratings = df_ratings.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
Ratings.head()

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
R = Ratings.values
#print(R)
user_ratings_mean = np.mean(R, axis = 1)
#print(user_ratings_mean.shape)
print(user_ratings_mean.size)
Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1) ## Making the user_ratings_mean vertical by reshaping

In [ ]:
sparsity = round(1.0 - len(ratings) / float(n_users * n_movies), 3)
print('The sparsity level of MovieLens100K dataset is ' +  str(sparsity * 100) + '%')

In [ ]:
from scipy.sparse.linalg import svds

In [ ]:
U, sigma, Vt = svds(Ratings_demeaned, k = 50)

In [ ]:
print('Size of sigma: ' , sigma.size)

In [ ]:
sigma = np.diag(sigma)

In [ ]:
print('Shape of sigma: ', sigma.shape)
print(sigma)

In [ ]:
print('Shape of U: ', U.shape)
print('Shape of Vt: ', Vt.shape)

In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [ ]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = Ratings.columns)
preds.head()

In [ ]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):
    """
    Implementation of SVD by hand
    :param predictions : The SVD reconstructed matrix, 
    userID : UserId for which you want to predict the top rated movies, 
    movies : Matrix with movie data, original_ratings : Original Rating matrix, 
    num_recommendations : num of recos to be returned
    :return: num_recommendations top movies
    """ 
    # Get and sort the user's predictions
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = predictions.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.userId == (userID)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [ ]:
already_rated, predictions = recommend_movies(preds, 150, movies, ratings, 20)

In [ ]:
# Top 20 movies that User 1310 has rated 
already_rated.head(20)

In [ ]:
# Top 20 movies that User 1310 hopefully will enjoy
predictions